In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from os import listdir

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/covidDetection /train'

bacterial_viral  COVID


In [ ]:
data_list_model2c = listdir('/content/drive/MyDrive/covidDetection /train')
data_list_model2c

['COVID', 'bacterial_viral']

In [ ]:
DATASET_PATH_model2c = '/content/drive/MyDrive/covidDetection /train'
train_dir_model2c = '/content/drive/MyDrive/covidDetection /train'
test_dir_model2c = '/content/drive/MyDrive/covidDetection /test'

In [ ]:
 IMAGE_SIZE = (150,150)
NUM_CLASSES = len(data_list_model2c)
BATCH_SIZE = 10
NUM_EPOCHS = 100
LEARNING_RATE = 0.0001

In [ ]:
 train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   featurewise_center=True,
                                   featurewise_std_normalization=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.25,
                                   zoom_range=0.1,
                                   zca_whitening = True,
                                   channel_shift_range=20,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='constant')


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1054: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn(


In [ ]:
 train_batches =train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                 target_size=IMAGE_SIZE,
                                                 shuffle=True,
                                                 batch_size=BATCH_SIZE,
                                                 subset = "training",
                                                 seed=42,
                                                 class_mode="categorical")

Found 175 images belonging to 2 classes.


In [ ]:
valid_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                 target_size=IMAGE_SIZE,
                                                 shuffle=True,
                                                 batch_size=BATCH_SIZE,
                                                 subset = "validation",
                                                 seed=42,
                                                 class_mode="categorical")


Found 0 images belonging to 2 classes.


In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(150,150,3)))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
 for layer in vgg16_model.layers[:-4]:
  layer.trainable = False
model2c = tf.keras.Sequential()
model2c.add(vgg16_model)
model2c.add(Flatten())
model2c.add(Dense(64,activation='relu'))
model2c.add(Dense(2,activation='softmax'))

In [ ]:
model2c.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN = train_batches.n//train_batches.batch_size
STEP_SIZE_VALID = valid_batches.n//valid_batches.batch_size


In [ ]:
model2c.fit(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs= 5, verbose=True)

/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:1286: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.6053 - loss: 0.7052
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7000 - loss: 0.4942
Epoch 3/5


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


17/17 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.8665 - loss: 0.3805
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 998us/step - accuracy: 0.9000 - loss: 0.2225
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.8742 - loss: 0.2423


In [ ]:
# Evaluate the model
trainResult = model2c.evaluate(
    TrainBatch,  # Generator object
    steps=len(TrainBatch),
    use_multiprocessing=False,
    verbose=1,
    workers=1
)
print('Train loss:', trainResult[0], 'Train accuracy:', trainResult[1])

# Predict using the model
prediction = model2c.predict(
    TrainBatch,  # Generator object
    steps=len(TrainBatch),
    verbose=1
)

# Convert predictions to class indices
Predicted_class = np.argmax(prediction, axis=1)

# Print confusion matrix
print(confusion_matrix(TrainBatch.classes, Predicted_class))


In [ ]:
Train_DataGeneration = ImageDataGenerator(rescale=1./255)
TrainBatch = Train_DataGeneration.flow_from_directory(train_dir_model2c,target_size=IMAGE_SIZE,batch_size=1,shuffle=False,seed=42,class_mode='categorical')
TrainBatch.reset()
# Evaluate the model
trainResult = model2c.evaluate(
    TrainBatch,  # Generator object
    steps=len(TrainBatch),
    verbose=1
)
print('Train loss:', trainResult[0], 'Train accuracy:', trainResult[1])

# Predict using the model
prediction = model2c.predict(
    TrainBatch,  # Generator object
    steps=len(TrainBatch),
    verbose=1
)

# Convert predictions to class indices
Predicted_class = np.argmax(prediction, axis=1)

# Print confusion matrix
print(confusion_matrix(TrainBatch.classes, Predicted_class))


Found 175 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


175/175 ━━━━━━━━━━━━━━━━━━━━ 55s 305ms/step - accuracy: 0.7000 - loss: 0.6339
Train loss: 0.39588266611099243 Train accuracy: 0.8171428442001343
175/175 ━━━━━━━━━━━━━━━━━━━━ 80s 456ms/step
[[ 43  32]
 [  0 100]]


In [ ]:
# Evaluate the model on the testing set
testResult = model2c.evaluate(
    TestingBatch,  # Generator object
    steps=len(TestingBatch),
    verbose=1
)
print('Test loss:', testResult[0], 'Test accuracy:', testResult[1])

# Make predictions
prediction = model2c.predict(
    TestingBatch,
    steps=len(TestingBatch)
)
Predicted_class = np.argmax(prediction, axis=1)

# Print confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(TestingBatch.classes, Predicted_class))


In [ ]:
Test_DataGeneration = ImageDataGenerator(rescale=1./255)
TestingBatch = Test_DataGeneration.flow_from_directory(test_dir_model2c,target_size=IMAGE_SIZE,batch_size=1,shuffle=False,seed=42,class_mode='categorical')
TestingBatch.reset()
# Evaluate the model on the testing set
testResult = model2c.evaluate(
    TestingBatch,  # Generator object
    steps=len(TestingBatch),
    verbose=1
)
print('Test loss:', testResult[0], 'Test accuracy:', testResult[1])

# Make predictions
prediction = model2c.predict(
    TestingBatch,
    steps=len(TestingBatch)
)
Predicted_class = np.argmax(prediction, axis=1)

# Print confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(TestingBatch.classes, Predicted_class))


Found 50 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 363ms/step - accuracy: 0.5243 - loss: 0.9946
Test loss: 0.5331205129623413 Test accuracy: 0.7200000286102295
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 336ms/step
[[11 14]
 [ 0 25]]
